# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Prunier  ->  M. Prunier  |  ['M. Prunier']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
R. Andrae  ->  R. Andrae  |  ['R. Andrae']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 71 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.12517


extracting tarball to tmp_2507.12517...

 done.


M. Prunier  ->  M. Prunier  |  ['M. Prunier']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']


...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.12965


extracting tarball to tmp_2507.12965...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.13278


extracting tarball to tmp_2507.13278...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.12517-b31b1b.svg)](https://arxiv.org/abs/2507.12517) | **Magnetic fields in the intracluster medium with TNG-Cluster: properties, morphology, and tangential anisotropy**  |
|| K. Lehle, et al. -- incl., <mark>M. Prunier</mark>, <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-07-18*|
|*Comments*| *submitted to A&A*|
|**Abstract**|            We characterize the magnetic field properties of 352 massive galaxy clusters from the TNG-Cluster magnetohydrodynamical cosmological simulation with a focus on central magnetic field morphology in cool-core (CC) vs non-cool-core (NCC) clusters. We present the central values and radial profiles of magnetic field strength and plasma parameter as a function of mass, cooling status and redshift. Compared to low-redshift observations, TNG-Cluster produces reasonable magnetic field amplitudes in the central regions of clusters spanning a range of 1-200 muG. We then discuss the main finding of this work: z=0 cool-core clusters have preferentially tangential magnetic fields at a characteristic scale of ~ 0.1 r500c. These strongly tangential field orientations are specific to CCs. In contrast, across the full cluster population, magnetic fields show isotropic configurations at all radii and redshifts. As individual halos grow, the evolution of their magnetic field topologies is diverse: tangential features can be short-lived, persist over large cosmological time-scales, or periodically appear, vanish, and reappear towards z=0. We discuss the underlying physics and possible physical scenarios to explain the origin of these structures. We argue that both AGN feedback-driven outflows, and merger-driven sloshing motions, cannot explain the population-wide tangential bias in magnetic field orientation. Instead, we propose that the trapping of internal gravity waves is responsible for the tangentially biased magnetic field topologies that we find in cool-core TNG-Cluster halos, due to the strong entropy gradient in these clusters.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.12532-b31b1b.svg)](https://arxiv.org/abs/2507.12532) | **A Spectroscopic Search for Dormant Black Holes in Low-Metallicity Binaries**  |
|| P. Nagarajan, et al. -- incl., <mark>K. El-Badry</mark>, <mark>J. Müller-Horn</mark>, <mark>R. Seeburger</mark>, <mark>H.-W. Rix</mark>, <mark>R. Andrae</mark> |
|*Appeared on*| *2025-07-18*|
|*Comments*| *39 pages, 11 figures, submitted to PASP*|
|**Abstract**|            The discovery of the massive black hole (BH) system Gaia BH3 in pre-release Gaia DR4 data suggests that wide BH binaries with luminous companions may be significantly overrepresented at low metallicities. Motivated by this finding, we have initiated a spectroscopic survey of low-metallicity stars exhibiting elevated RUWE values in Gaia DR3, using the FEROS and APF spectrographs. We identify promising BH binary candidates as objects with instantaneously measured radial velocities (RVs) that are very different from their mean RVs reported in Gaia DR3. Thus far, we have observed over 500 targets, including a nearly complete sample of stars with $\text{[Fe/H]} < -1.5$, RUWE $> 2$, and $G < 15$. Our search has yielded one promising target exhibiting slow acceleration and an RV more than 98 km s$^{-1}$ different from its DR3 mean RV, as well as dozens of other candidates with smaller RV discrepancies. We quantify the sensitivity of our search using simulations, demonstrating that it recovers at least half of the BH companions within our selection criteria. We make all the spectra and RVs from our survey publicly available and encourage further follow-up.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.12876-b31b1b.svg)](https://arxiv.org/abs/2507.12876) | **Einstein Probe Discovery of EP J182730.0-095633: A New Black Hole X-ray Binary Candidate in Faint Outburst?**  |
|| H. Cheng, et al. -- incl., <mark>Y. Wang</mark>, <mark>M. Zhang</mark> |
|*Appeared on*| *2025-07-18*|
|*Comments*| *22 pages, 5 figures (plus 3 in appendix), 3 tables in appendix. Accepted for publication in ApJ Letters*|
|**Abstract**|            Black hole X-ray binaries (candidates) currently identified in our galaxy are mainly transient sources, with the majority discovered through the detection of their X-ray outbursts. Among these, only four were found during faint outbursts exhibiting peak X-ray luminosities $L_{\rm X}\lesssim10^{36}~{\rm erg~s^{-1}}$, likely due to the previous lack of sensitive, wide-field monitoring instruments in the X-ray band. In this Letter, we present the discovery of an intriguing X-ray transient, EP J182730.0-095633, via the Einstein Probe (EP) and subsequent multi-wavelength follow-up studies. This transient, located on the Galactic plane, experienced a faint and brief X-ray outburst lasting about 20 days. Its X-ray spectrum is non-thermal and consistent with a power-law model with a nearly constant photon index of $\Gamma \sim2$ throughout the outburst. A long-lasting millihertz quasi-periodic oscillation (QPO) signal was detected in its X-ray light curve, centered around a frequency of $\sim0.04$ Hz. A transient near-infrared source was identified as its counterpart, although no optical emission was detectable, likely due to significant extinction. A radio counterpart was also observed, displaying an inverted radio spectrum with $\alpha\sim0.45$. The X-ray spectral and temporal characteristics, along with the multi-wavelength properties, indicate that the source is a faint low-mass X-ray binary, with the compact object likely being a black hole. This work demonstrates the potential of the EP in discovering new X-ray binaries by capturing faint-level X-ray outbursts.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.12965-b31b1b.svg)](https://arxiv.org/abs/2507.12965) | **Gravitational wave standard sirens: A brief review of cosmological parameter estimation**  |
|| S.-J. Jin, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-18*|
|*Comments*| *36 pages, 20 figures*|
|**Abstract**|            Gravitational wave (GW) observations are expected to serve as a powerful and independent probe of the expansion history of the universe. By providing direct and calibration-free measurements of luminosity distances through waveform analysis, GWs provide a fundamentally different and potentially more robust approach to measuring cosmic-scale distances compared to traditional electromagnetic observations, which is known as the standard siren method. In this review, we present an overview of recent developments in GW standard siren cosmology, the latest observational results, and prospects for constraining cosmological parameters using future GW detections. We first introduce standard sirens based on how redshift information is obtained and outline the Bayesian framework used in cosmological parameter estimation. We then review the measurements on the Hubble constant from the LIGO-Virgo-KAGRA network and present the potential role of future standard siren observations in cosmological parameter estimations. A central focus of this review is the unique ability of GW observations to break cosmological parameter degeneracies inherent in the EM observations. Since the cosmological parameter degeneracy directions of GW and EM observations are quite different (roughly orthogonal in some cases), their combination can significantly improve constraints on cosmological parameters. This complementarity is expected to become one of the most critical advantages for GW standard siren cosmology. Looking forward, we highlight the importance of combining GW standard sirens with other emerging late-universe cosmological probes such as fast radio bursts, 21 cm intensity mapping, and strong gravitational lensing to forge a precise cosmological probe for exploring the late universe. Finally, we introduce the challenges and the role of machine learning in future standard siren analysis.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.13278-b31b1b.svg)](https://arxiv.org/abs/2507.13278) | **The pulsar wind nebula around B1853+01 in X-rays**  |
|| <mark>X. Zhang</mark>, P. Bordas, S. Safi-Harb, K. Iwasawa |
|*Appeared on*| *2025-07-18*|
|*Comments*| **|
|**Abstract**|            We report on the results of a comprehensive analysis of X-ray observations carried out with \textit{Chandra}, \textit{XMM-Newton} and \textit{NuSTAR} of the pulsar wind nebula (PWN) associated with PSR B1853+01, located inside the W44 supernova remnant (SNR). Previous X-ray observations unveiled the presence of a fast-moving pulsar, PSR B1853+01, located at the southern edge of the W44 thermal X-ray emission region, as well as an elongated tail structure trailing the pulsar. Our analysis reveals, in addition, the presence of an ``outflow'' feature ahead of the pulsar extending for about 1 \arcmin ($\sim$ 1.0 pc at the distance of 3.2 kpc). At larger scales, the entire PWN seems to be surrounded by a faint, diffuse X-ray emission structure. The southern part of this structure displays the same unusual morphology as the ``outflow'' feature ahead of the pulsar, and extends along $\sim 6$ \arcmin ($\sim$ 5 pc) in the direction of the pulsar proper motion. In this report, a spatially-resolved spectral analysis for the different extended regions around PSR B1853+01 is carried out. For an updated value of the column density of $0.65_{+0.46}^{-0.42} \times 10^{22} ~\textrm{cm}^{-2}$, a power-law fit to the ``outflow'' region yields a spectral index $\Gamma \approx 1.24_{+0.23}^{-0.24}$, which is significantly harder than that of the pulsar ($\Gamma \approx 1.87_{+0.48}^{-0.43}$) and the pulsar tail ($\Gamma \approx 2.01_{+0.39}^{-0.38}$). We argue that both the ``outflow'' structure and the surrounding halo-like X-ray emission might be produced by high-energy particles escaping the PWN around PSR B1853+01, a scenario recently suggested also for other Bow-shock PWNe with jet-like structures and/or TeV halos.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.12517/./figures/StreamlinePlots/Halo_17440255_lic_betaA_0p4r500_sn99.png', 'tmp_2507.12517/./figures/Anisotropy/betaProfile_0tor500_CCness.png', 'tmp_2507.12517/./figures/Bfields/CoreGallery_B_sort.png', 'tmp_2507.12517/./figures/Bfields/BetaPprofile_z0_SCCvsNCC.png', 'tmp_2507.12517/./figures/Bfields/Projection_Pth_Halo17440255.png', 'tmp_2507.12517/./figures/Bfields/Projection_Pmag_Halo17440255.png']
copying  tmp_2507.12517/./figures/StreamlinePlots/Halo_17440255_lic_betaA_0p4r500_sn99.png to _build/html/
copying  tmp_2507.12517/./figures/Anisotropy/betaProfile_0tor500_CCness.png to _build/html/
copying  tmp_2507.12517/./figures/Bfields/CoreGallery_B_sort.png to _build/html/
copying  tmp_2507.12517/./figures/Bfields/BetaPprofile_z0_SCCvsNCC.png to _build/html/
copying  tmp_2507.12517/./figures/Bfields/Projection_Pth_Halo17440255.png to _build/html/
copying  tmp_2507.12517/./figures/Bfields/Projection_Pmag_Halo17440255.png to _build/html/
exported in  _build/h

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mvir}{{\rm M}_{\rm 500c}}$
$\newcommand{\rvir}{r_{\rm 500c}}$
$\newcommand{\mtc}{{\rm M}_{\rm 200c}}$
$\newcommand{\rtc}{r_{\rm 200c}}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\betaP}{\beta_{\rm P}}$
$\newcommand{\betaA}{\beta_{\rm A}}$
$\newcommand{\cmr}[1]{{\color{red}\bf{[MR: #1]}}}$
$\newcommand{\mr}[1]{{\color{red}#1}}$</div>



<div id="title">

# Magnetic fields in the intracluster medium with TNG-Cluster: properties, morphology, and tangential anisotropy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.12517-b31b1b.svg)](https://arxiv.org/abs/2507.12517)<mark>Appeared on: 2025-07-18</mark> -  _submitted to A&A_

</div>
<div id="authors">

K. Lehle, et al. -- incl., <mark>M. Prunier</mark>, <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** We characterize the magnetic field properties of 352 massive galaxy clusters from the TNG-Cluster magnetohydrodynamical cosmological simulation with a focus on central magnetic field morphology in cool-core (CC) vs non-cool-core (NCC) clusters. We present the central values and radial profiles of magnetic field strength and plasma parameter as a function of mass, cooling status and redshift. Compared to low-redshift observations, TNG-Cluster produces reasonable magnetic field amplitudes in the central regions of clusters spanning a range of $1-200  \mu$ G. We then discuss the main finding of this work: $z=0$ cool-core clusters have preferentially tangential magnetic fields at a characteristic scale of $\sim 0.1 \rvir$ . These strongly tangential field orientations are specific to CCs. In contrast, across the full cluster population, magnetic fields show isotropic configurations at all radii and redshifts. As individual halos grow, the evolution of their magnetic field topologies is diverse: tangential features can be short-lived, persist over large cosmological time-scales, or periodically appear, vanish, and reappear towards $z=0$ . We discuss the underlying physics and possible physical scenarios to explain the origin of these structures. We argue that both AGN feedback-driven outflows, and merger-driven sloshing motions, cannot explain the population-wide tangential bias in magnetic field orientation. Instead, we propose that the trapping of internal gravity waves is responsible for the tangentially biased magnetic field topologies that we find in cool-core TNG-Cluster halos, due to the strong entropy gradient in these clusters.

</div>

<div id="div_fig1">

<img src="tmp_2507.12517/./figures/StreamlinePlots/Halo_17440255_lic_betaA_0p4r500_sn99.png" alt="Fig7.1" width="50%"/><img src="tmp_2507.12517/./figures/Anisotropy/betaProfile_0tor500_CCness.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** The topology of magnetic fields can be preferentially tangential for cool-core clusters. **Top panel:** Visualization of magnetic fields in a single TNG-Cluster halo at $z=0$(Halo 17440255) showing the central $40\%$ of $\rvir$. The background color shows the magnetic anisotropy $\betaA$ in the $xy$-plane. Negative values (yellow) correspond to tangential magnetic fields and positive values (blue) correspond to radially oriented magnetic fields. Overplotted in gray are the magnetic field directions, which appear as relief-like structures in the image. The white circle marks the radius at which the magnetic anisotropy profile (bottom panel) has its minimum. **Bottom panel:** Radial profiles of magnetic field anisotropy parameter $\betaA$ at $z=0$ for all halos in TNG-Cluster. Thin lines show individual halos, colored by CC state. The thick lines show the median profiles for SCCs (blue), WCCs (purple) and NCCs (orange). WCCs and NCCs show a flat profile at $\betaA\sim0$, while SCCs exhibit a dip at $\sim 0.1$\rvir$$ with $\betaA <0$. This shows that WCCs and NCCs have primarily isotropic magnetic fields, while SCCs have preferably tangentially oriented magnetic fields in the vicinity of this radius (as indicated by black arrow). (*fig:betaA*)

</div>
<div id="div_fig2">

<img src="tmp_2507.12517/./figures/Bfields/CoreGallery_B_sort.png" alt="Fig3" width="100%"/>

**Figure 3. -** Gallery of magnetic field strength and morphology in sixteen high-mass galaxy clusters selected from TNG-Cluster at $z=0$. Each panel shows the central region and extends $0.2$\rvir$$ from side-to-side, giving projections of mean magnetic field strength in a thin slice of $15$ kpc depth. The white circles indicate $0.1$\rvir$$. The panels are ordered by ascending central entropy $K_0$, such that the strongest cool-core (CC) systems are in the upper left, and the strongest non-cool-core (NCC) halos are located in the bottom right. Central magnetic field strength increases rapidly with halo mass, and is higher in CC versus NCC clusters. (*fig:galleryB*)

</div>
<div id="div_fig3">

<img src="tmp_2507.12517/./figures/Bfields/BetaPprofile_z0_SCCvsNCC.png" alt="Fig5.1" width="33%"/><img src="tmp_2507.12517/./figures/Bfields/Projection_Pth_Halo17440255.png" alt="Fig5.2" width="33%"/><img src="tmp_2507.12517/./figures/Bfields/Projection_Pmag_Halo17440255.png" alt="Fig5.3" width="33%"/>

**Figure 5. -** **Main panel:** radial profiles of the plasma beta parameter $\beta_{\rm P}$ at $z=0$ for all TNG-Cluster halos, colored according to CC state. Thin lines show individual clusters, while the three thick lines show median stacks of SCC, WCC, and NCC clusters. **Lower panels:** maps of thermal pressure (left) and magnetic pressure (right) of one individual SCC halo ans an example. Both maps show the central region with $r<0.2$\rvir$$ for a slice with depth $15$ kpc. The overall pressure budget is dominated by the thermal pressure for both CCs and NCCs. $\beta_{\rm P}$ profiles for WCCs and NCCs are flat with average values of $\beta_{\rm P}\sim 900$. SCCs have a small dip in the core and a excess at $0.1 $\rvir$$. However, these trends are rather weak. (*fig:profileBetaP*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.12517"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# A Spectroscopic Search for Dormant Black Holes in Low-Metallicity Binaries

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.12532-b31b1b.svg)](https://arxiv.org/abs/2507.12532)<mark>Appeared on: 2025-07-18</mark> -  _39 pages, 11 figures, submitted to PASP_

</div>
<div id="authors">

P. Nagarajan, et al. -- incl., <mark>K. El-Badry</mark>, <mark>J. Müller-Horn</mark>, <mark>R. Seeburger</mark>, <mark>H.-W. Rix</mark>, <mark>R. Andrae</mark>

</div>
<div id="abstract">

**Abstract:** The discovery of the massive black hole (BH) system Gaia BH3 in pre-release _Gaia_ DR4 data suggests that wide BH binaries with luminous companions may be significantly overrepresented at low metallicities. Motivated by this finding, we have initiated a spectroscopic survey of low-metallicity stars exhibiting elevated \texttt{RUWE} values in $_ Gaia_$ DR3, using the FEROS and APF spectrographs. We identify promising BH binary candidates as objects with instantaneously measured radial velocities (RVs) that are very different from their mean RVs reported in _Gaia_ DR3. Thus far, we have observed over 500 targets, including a nearly complete sample of stars with $\rm[Fe/H] < -1.5$ , \texttt{RUWE} $> 2$ , and $G < 15$ . Our search has yielded one promising target exhibiting slow acceleration and an RV more than 98 km s $^{-1}$ different from its DR3 mean RV, as well as dozens of other candidates with smaller RV discrepancies. We quantify the sensitivity of our search using simulations, demonstrating that it recovers at least half of the BH companions within our selection criteria. We make all the spectra and RVs from our survey publicly available and encourage further follow-up.

</div>

<div id="div_fig1">

<img src="tmp_2507.12532/./RUWE_Trend_Figure.png" alt="Fig2" width="100%"/>

**Figure 2. -** Dependence of predicted \texttt{RUWE} in DR3 on orbital period, distance, and BH mass for low-metallicity dwarf or giant companions to BHs, with the shaded regions representing a 68\% confidence interval. We randomly sample sky positions from our catalog of candidates, orbital eccentricities from a thermal distribution, and isotropic binary orientations. At orbital periods of $10^2$--$10^3$ d, \texttt{RUWE} increases as the projected semi-major axis of the photocenter increases, peaking at $\sim10^3$ days. At orbital periods $> 10^3$ d, the limited orbital coverage during _Gaia_ DR3's $\sim1000$ d observing baseline causes \texttt{RUWE} to decrease. \texttt{RUWE} also decreases as the binary gets farther away, since the projected semi-major axis of the photocenter decreases. \texttt{RUWE} is only weakly sensitive to BH mass. Due to their brighter absolute magnitudes, giants tend to have slightly larger \texttt{RUWE} values than dwarfs, and are detectable out to farther distances. With a selection cut of \texttt{RUWE}$> 2.0$, we are sensitive to orbital periods of $100$--$5000$ d and distances out to $\sim2$ kpc. (*fig:ruwe_trend_fig*)

</div>
<div id="div_fig2">

<img src="tmp_2507.12532/./Compare_Gaia_RV_Figure.png" alt="Fig6" width="100%"/>

**Figure 6. -** Comparison of our derived RVs to the mean RV reported in DR3. RVs for flagged spectra (see Section \ref{sec:problematic}) are plotted in orange. In most cases, the RVs are in good agreement. In several cases, the RVs are discrepant, suggesting the presence of a faint or dark companion. We highlight both Gaia BH3 and _Gaia_ DR3 6769569470180424704, which has the largest RV discrepancy in our sample. (*fig:compare_gaia_rvs*)

</div>
<div id="div_fig3">

<img src="tmp_2507.12532/./Gaia_BH3_RV_Astrometry.png" alt="Fig1" width="100%"/>

**Figure 1. -** Simulated _Gaia_ DR3 epoch astrometry and radial velocity curve for Gaia BH3 based on best-fit combined parameters from gaia_collaboration_discovery_2024. In the left panel, the gray line connects the barycenter to periastron. In the right panel, the temporal range of the simulated DR3 epoch astrometry is plotted with higher opacity. The epoch RVs of Gaia BH3 are different from its reported mean RV by at least 20 km s$^{-1}$ over $\sim70\%$ of its orbit (shaded purple region), suggesting that a search strategy based on this comparison may be useful in finding similar targets. (*fig:bh3_rv_astro*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.12532"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

145  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

17  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
